In [1]:
# pull in quality data from Wikimedia
from paws.EpochFail import wikiquality as wq

In [2]:
# check to make sure we got what we expected
print(next(wq.read_rows('en')))

{'weighted_sum': 0.10413085831157592, 'title': 'Lost Moon', 'timestamp': '20051201000000', 'rev_id': 28847753, 'prediction': 'Stub', 'page_id': 3204489}


In [3]:
# here's what those 'predicted' values mean: 
# https://meta.wikimedia.org/wiki/Objective_Revision_Evaluation_Service#Article_quality_models


In [9]:
# look for the women scientists
print(next(wq.read_page_ids('women scientists')))

{'page_id': 9622178}


In [32]:
import json
f = open('data.txt')
for i in(wq.read_page_ids('women scientists')):
    f.write(str(i['page_id']))

UnsupportedOperation: not writable

In [ ]:
import sys
from pprint import pprint
from math import log, exp
from collections import defaultdict

from paws.EpochFail import wikiquality

women_scientists = set(r['page_id'] for r in wikiquality.read_page_ids('women scientists'))

ws_wiki_quality = {}

women_scientist_aq = (row for row in wikiquality.read_aq('en') if row['page_id'] in women_scientists)

for i, row in enumerate(women_scientist_aq):
    if row['timestamp'] not in monthly_wiki_quality:
        monthly_wiki_quality[row['timestamp']] = {
            'weighted_sum': 0.0, 
            'weighted_log_sum': 0.0, 
            'n': 0, 
            'predictions': defaultdict(int)
        }
    monthly_wiki_quality[row['timestamp']]['n'] += 1
    monthly_wiki_quality[row['timestamp']]['weighted_sum'] += row['weighted_sum']
    monthly_wiki_quality[row['timestamp']]['weighted_log_sum'] += log(row['weighted_sum'])
    monthly_wiki_quality[row['timestamp']]['predictions'][row['prediction']] += 1
    if i % 10 == 0:
        sys.stderr.write(".")
    if i != 0 and i % 10**3 == 0:
        sys.stderr.write("\n")

In [71]:
def pageQual(page_id_set):
    import sys
    from pprint import pprint
    from math import log, exp
    from collections import defaultdict

    from paws.EpochFail import wikiquality as wq
    pageByMonth = {} 
    while len(pageByMonth) < 3:
        for i in wq.read_aq('en'):
            if i['page_id'] in page_id_set:
                if i['page_id'] not in pageByMonth:
                    pageByMonth[i['page_id']] = {}
                time = i['timestamp']
                newVal = dict(i)
                del newVal['timestamp']
                del newVal['page_id']
                pageByMonth[i['page_id']][time] = newVal
            
    return pageByMonth
                
def getPageIDs(category, file_name):
    page_id_set = set()
    generator = wq.read_page_ids(category)
    for page in generator:
        page_id_set.add(page['page_id'])
    with open(file_name, 'w') as f:
        for page_id in page_id_set:
            f.write(str(page_id) + '\n')
    return page_id_set

In [61]:
women_scientists = getPageIDs('women scientists', 'data.txt')

In [72]:
pageQual(women_scientists)

KeyboardInterrupt: 